In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


/Users/rohitramaswamy/Desktop/programming-stuff/sports-data-scraping/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


In [3]:

# Initialize current_df correctly
current_df = pd.DataFrame(columns=["Player (age)"])

year = 1970
for i in range(2024-year+1):
    url = f'https://www.pro-football-reference.com/leaders/rush_yds_career_{year}.htm'
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check if the request was successful

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all tables in the page
        tables = soup.find_all('table')

        # Convert the first table to a pandas DataFrame
        if tables:
            df = pd.DataFrame(pd.read_html(str(tables[0]))[0])
            # if 'Player' in df.columns and 'Runs' in df.columns:
            df = df[['Player (age)', 'Yds']].head(50)
                # df['Country'] = df['Player'].str.split('(').str[1].str.strip(')')
                # df['Player'] = df['Player'].str.split('(').str[0]

            df.rename(columns={'Yds': f'{year}'}, inplace=True)
            current_df = pd.merge(current_df, df, on='Player (age)', how='outer')
                # current_df['Country'] = current_df['Country_x'].combine_first(current_df['Country_y'])
                # current_df = current_df.drop(columns=['Country_x', 'Country_y'])

            # else:
            #     print(f"'Player' or 'Runs' column not found for the year {year}")
        else:
            print("No tables found on the page.")
    except requests.HTTPError as e:
        print(f"HTTP error occurred: {e.response.status_code} - {e.response.reason}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    year = year + 1

current_df = current_df.fillna(0)


print(current_df)



An error occurred: HTTPSConnectionPool(host='www.pro-football-reference.com', port=443): Max retries exceeded with url: /leaders/rush_yds_career_1970.htm (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x123131a60>: Failed to resolve 'www.pro-football-reference.com' ([Errno 8] nodename nor servname provided, or not known)"))
An error occurred: HTTPSConnectionPool(host='www.pro-football-reference.com', port=443): Max retries exceeded with url: /leaders/rush_yds_career_1971.htm (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x123134b20>: Failed to resolve 'www.pro-football-reference.com' ([Errno 8] nodename nor servname provided, or not known)"))
An error occurred: HTTPSConnectionPool(host='www.pro-football-reference.com', port=443): Max retries exceeded with url: /leaders/rush_yds_career_1972.htm (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x123164490>: Failed to resolve 'www.pro-football-refe

In [68]:

# List of countries
countries = pd.read_csv('shortcodes.csv', dtype=str)  

print(countries.tail(5))

# row_index = countries[countries["Code"] == 'IND'].index[0]
# print(row_index)

# Function to get Shortcode based on Code
def get_row(input):
    rows = countries[countries["Code"] == input].index
    if len(rows) > 0:
        row_index = rows[0]
        if len(countries.loc[row_index, "Lowercase shortcode"]) == 2:
            return f'https://public.flourish.studio/country-flags/svg/{countries.loc[row_index, "Lowercase shortcode"]}.svg'
        else:
            return countries.loc[row_index, "Lowercase shortcode"]
    else:
        return 'Unknown'  # Or any default value you want to assign

# Apply function to get Shortcode for each country in current_df
current_df["Flag"] = current_df["Country"].apply(get_row)

print(current_df.head(5))

            Name  Code Shortcode  \
245        Yemen    YE      :YE:   
246       Zambia    ZM      :ZM:   
247     Zimbabwe   ZIM     :ZIM:   
248  West Indies    WI       NaN   
249     Scotland  SCOT       NaN   

                                   Lowercase shortcode  
245                                                 ye  
246                                                 zm  
247                                                 zw  
248  https://upload.wikimedia.org/wikipedia/en/9/9b...  
249  https://upload.wikimedia.org/wikipedia/commons...  
            Player 2005  2006   2007   2008   2009   2010   2011   2012  \
0     A Balbirnie     0   0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1      A Flintoff     6   0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2       A Symonds    32  86.0  125.0  337.0  337.0  337.0  337.0    0.0   
3  AB de Villiers     0   0.0    0.0  133.0  362.0  579.0  604.0  680.0   
4    AC Gilchrist    16  17.0   65.0  272.0  272.0    0.0    0.0  

In [17]:
excel_file = "nfl_output.xlsx" 
 
# Write DataFrame to Excel 
current_df.to_excel(excel_file, index=False) 